### Challenge 23: Clone an MT19937 RNG from its output

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

<div class="alert alert-block alert-info">

The internal state of MT19937 consists of 624 32 bit integers.

For each batch of 624 outputs, MT permutes that internal state. By permuting state regularly, MT19937 achieves a period of 2**19937, which is Big.

Each time MT19937 is tapped, an element of its internal state is subjected to a tempering function that diffuses bits through the result.

The tempering function is invertible; you can write an "untemper" function that takes an MT19937 output and transforms it back into the corresponding element of the MT19937 state array.

To invert the temper transform, apply the inverse of each of the operations in the temper transform in reverse order. There are two kinds of operations in the temper transform each applied twice; one is an XOR against a right-shifted value, and the other is an XOR against a left-shifted value AND'd with a magic number. So you'll need code to invert the "right" and the "left" operation.

Once you have "untemper" working, create a new MT19937 generator, tap it for 624 outputs, untemper each of them to recreate the state of the generator, and splice that state into a new instance of the MT19937 generator.

The new "spliced" generator should predict the values of the original.
    
<div class="alert alert-block alert-warning">

### Stop and think for a second.
    
How would you modify MT19937 to make this attack hard? What would happen if you subjected each tempered output to a cryptographic hash?
    
</div>

</div>

In [3]:
import mt19937 as mt
from random import randint

In [4]:
# Get some outputs from RNGs seeded with current system time in the recent past

seed = randint(0, 2**32-1)  # Randomly select seed from all 4 billion possible
myRNG = mt.mt19937(seed)
observed_out = [0]*624


for ii in range(624):
    observed_out[ii] = myRNG.extract_number()

To do this...we need to be able to reverse the temper step of the Marsenne Twister.  

This one is fun.  Here's my solution:

```python
def mt_reverse_temper(y):

    # Undo:  y ^= (y >> cL)
    y ^= (y >> cL) ^ (y >> (cL*2))

    # Undo:  y ^= ((y << cT) & cC)
    y ^= (y << cT) & cC

    # Undo:  y ^= ((y << cS) & cB)
    x = y
    for ii in range(4):
        x = (x << cS) & cB
        y ^= x

    # Undo:  y ^= ((y >> cU) & cD)
    y ^= ((y >> cU) ^ (y >> (cU * 2)))

    return(y & cD)
```

In [6]:
guessed_state = [0]*624
for ii in range(624):
    # Figure out what internal state value would have generated this output
    guessed_state[ii] = mt.mt_reverse_temper(observed_out[ii])

clonedRNG = mt.mt19937(0)
clonedRNG.state = guessed_state
# Don't set index...let it do a twist to sync up

# Let's try it out...predict next RNG output w/ clonedRNG, then get the actual next output
# Let it go through a twist to see if it stays synced up
for ii in range(1000):
    print(f"Predicted /Actual Number:  {clonedRNG.extract_number()}, {myRNG.extract_number()}")

Predicted /Actual Number:  835382483, 1898116435
Predicted /Actual Number:  4285797119, 2507699207
Predicted /Actual Number:  3346485199, 3137645264
Predicted /Actual Number:  3908370743, 1172309501
Predicted /Actual Number:  1287335748, 4080713854
Predicted /Actual Number:  2624300160, 3690863853
Predicted /Actual Number:  3134420965, 836167810
Predicted /Actual Number:  1234564817, 704344388
Predicted /Actual Number:  3188670476, 1566126065
Predicted /Actual Number:  2201282504, 871411404
Predicted /Actual Number:  1092640993, 816773437
Predicted /Actual Number:  2578171189, 1539855810
Predicted /Actual Number:  2117953811, 430282278
Predicted /Actual Number:  2494797219, 3319679624
Predicted /Actual Number:  3190316203, 3047776808
Predicted /Actual Number:  104973518, 2320395416
Predicted /Actual Number:  1378218901, 2913962109
Predicted /Actual Number:  1435979148, 1199534128
Predicted /Actual Number:  3677162314, 3489705400
Predicted /Actual Number:  3176044373, 3182854102
Predict

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)